## 加载向量数据库

In [1]:
import sys
# sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中

# 使用智谱 Embedding API，注意，需要将上一章实现的封装代码下载到本地
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.vectorstores.chroma import Chroma

In [2]:
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

In [3]:
# 定义 Embeddings
embedding = ZhipuAIEmbeddings()

# 向量数据库持久化路径
persist_directory = '../data_base/vector_db/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)

In [4]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：10


In [5]:
question = "什么是prompt engineering?"
docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：3


In [8]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

检索到的第0个内容: 
 第二章 提示原则

如何去使用 Prompt，以充分发挥 LLM 的性能？首先我们需要知道设计 Prompt 的原则，它们是每一个开发者设计 Prompt 所必须知道的基础概念。本章讨论了设计高效 Prompt 的两个关键原则：编写清晰、具体的指令和给予模型充足思考时间。掌握这两点，对创建可靠的语言模型交互尤为重要。

首先，Prompt 需要清晰明确地表达需求，提供充足上下文，使语言模型准确理解我们的意图，就像向一个外星人详细解释人类世界一样。过于简略的 Prompt 往往使模型难以把握所要完成的具体任务。

其次，让语言模型有充足时间推理也极为关键。就像人类解题一样，匆忙得出的结论多有失误。因此 Prompt 应加入逐步推理的要求，给模型留出充分思考时间，这样生成的结果才更准确可靠。

如果 Prompt 在这两点上都作了优化，语言模型就能够尽可能发挥潜力，完成复杂的推理和生成任务。掌握这些 Prompt 设计原则，是开发者取得语言模型应用成功的重要一步。

一、原则一 编写清晰、具体的指令
-----------------------------------------------------
检索到的第1个内容: 
 一、原则一 编写清晰、具体的指令

亲爱的读者，在与语言模型交互时，您需要牢记一点:以清晰、具体的方式表达您的需求。假设您面前坐着一位来自外星球的新朋友，其对人类语言和常识都一无所知。在这种情况下，您需要把想表达的意图讲得非常明确，不要有任何歧义。同样的，在提供 Prompt 的时候，也要以足够详细和容易理解的方式，把您的需求与上下文说清楚。

并不是说 Prompt 就必须非常短小简洁。事实上，在许多情况下，更长、更复杂的 Prompt 反而会让语言模型更容易抓住关键点，给出符合预期的回复。原因在于，复杂的 Prompt 提供了更丰富的上下文和细节，让模型可以更准确地把握所需的操作和响应方式。

所以，记住用清晰、详尽的语言表达 Prompt，就像在给外星人讲解人类世界一样，“Adding more context helps the model understand you better.”。

从该原则出发，我们提供几个设计 Prompt 的技巧。

1.1 使用分隔符清晰地表示输入的不同部分
--------

## 创建一个LLM

In [9]:
import os 
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [10]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)

llm.invoke("请你自我介绍一下自己！")

AIMessage(content='你好！我是一个AI助手，专门为用户提供各种服务和帮助。我可以回答各种问题，提供信息和建议，还可以进行语言翻译和文本编辑等任务。我还在不断学习和进化，以提供更好的服务。如果你有任何问题或需要帮助，请随时告诉我！', response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 20, 'total_tokens': 122}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d9b51504-2977-4844-8096-e82d03a4bc11-0')

## 构建检索问答链

In [11]:
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)

In [14]:
QA_CHAIN_PROMPT

PromptTemplate(input_variables=['context', 'question'], template='使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答\n案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。\n{context}\n问题: {question}\n')

再创建一个基于模板的检索链：

In [15]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

创建检索 QA 链的方法 RetrievalQA.from_chain_type() 有如下参数：
- llm：指定使用的 LLM
- 指定 chain type : RetrievalQA.from_chain_type(chain_type="map_reduce")，也可以利用load_qa_chain()方法指定chain type。
- 自定义 prompt ：通过在RetrievalQA.from_chain_type()方法中，指定chain_type_kwargs参数，而该参数：chain_type_kwargs = {"prompt": PROMPT}
- 返回源文档：通过RetrievalQA.from_chain_type()方法中指定：return_source_documents=True参数；也可以使用RetrievalQAWithSourceChain()方法，返回源文档的引用（坐标或者叫主键、索引）

## 检索问答链效果测试

In [16]:
question_1 = "什么是南瓜书？"
question_2 = "王阳明是谁？"

In [17]:
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

d:\software\miniconda3\envs\llm-universe\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


大模型+知识库后回答 question_1 的结果：
我不知道什么是南瓜书。谢谢你的提问！


In [18]:
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

大模型+知识库后回答 question_2 的结果：
我不知道王阳明是谁。谢谢你的提问！


### 4.2 大模型自己回答的效果

In [19]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
llm.predict(prompt_template)

d:\software\miniconda3\envs\llm-universe\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"南瓜书是指《深入理解计算机系统》（Computer Systems: A Programmer's Perspective）一书的俗称。这本书是由Randal E. Bryant和David R. O'Hallaron合著的计算机科学教材，旨在帮助读者深入理解计算机系统的工作原理和底层机制。南瓜书因其封面上有一个南瓜图案而得名，被广泛用于大学的计算机科学和工程课程中。"

In [20]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
llm.predict(prompt_template)

'王阳明（1472年-1529年），字仲明，号阳明子，是明代中期著名的思想家、政治家、军事家和教育家。他提出了“心即理”、“知行合一”的思想，强调人的内心自觉和道德修养的重要性。他的思想对中国历史产生了深远的影响，被后世尊称为“阳明先生”。'

## 5. 添加历史对话的记忆功能

In [21]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

## 2. 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

In [22]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)
question = "我可以学习到关于提示工程的知识吗？"
result = qa({"question": question})
print(result['answer'])

是的，您可以通过本模块学习关于提示工程的知识。本模块将与您分享使用提示词开发大语言模型应用的最佳实践和技巧。内容涵盖了软件开发提示词设计、文本总结、推理、转换、扩展以及构建聊天机器人等语言模型典型应用场景。我们希望通过这些内容能够激发您的想象力，帮助您开发出更出色的语言模型应用。


In [23]:
question = "为什么这门课需要教这方面的知识？"
result = qa({"question": question})
print(result['answer'])

这门课程需要教授关于提示工程的知识，是因为提示工程对于开发者来说非常重要。在使用大语言模型（LLM）进行开发时，设计清晰、具体的提示是确保模型能够准确理解开发者意图的关键。通过学习提示工程的知识，开发者可以了解如何设计高效的提示，以充分发挥LLM的性能，并创建可靠的语言模型交互。提示工程的技巧和最佳实践可以帮助开发者提升LLM应用的效果，实现更出色的语言模型应用。
